## 讯飞农作物生长情况识别挑战赛baseline(非官方)



### 一、赛事背景
随着物联网和大数据技术的飞速发展，智慧农业已经是智慧经济的重要组成部分。我国是农业大国，而非农业强国，因此要实现高水平的设施农业生产和优化设施生物环境控制，农业相关信息获取、分析技术是农业生产中最关键的技术之一。

农作物的生长态势，事关农事生产的整个过程，因此通过农作物不同时期图片，对农作物进行合理的农作物生产态势检测，对于农业生产是十分有必要的。通过分析农作物生长情况，最大程度地判断农作物生长态势，合理调配生产资源，为农作物生产管理人员或管理决策者提供及时可靠的长势信息，便于及时采集有效的田间管理措施，对农作物产量进行准确预估，为我国人民的生存条件和粮食安全提供保障。

### 二、赛事任务
通过作物不同生长时期的特点可以对作物的生长情况进行识别，给出合理的作物生长阶段。本次大赛提供了大量草莓植株在营养生长阶段的生长情况图片作为样本，参赛选手需基于提供的样本构建模型，对草莓样本生长态势进行检测，判断其生长情况，并将生长情况在csv文件中对应标定出来，给出草莓图像对应的生长阶段。

初赛一阶段：6月21日到8月18日。

初赛二阶段：8月19日到8月20日。系统将在8月19日00：00更换测试数据，参赛队伍需要再次下载数据。

### 三、评审规则
1.数据说明
本次比赛将会为选手提供草莓处于营养生长阶段的图片作为数据集，其中包含有作物图片及生长情况标签。选手根据训练集进行训练，对测试集数据进行标定，判断所标定的作物处于何种生长情况之下。草莓生长阶段大致可以分为以下几个阶段：生长期、开花期、结果期、成熟期。

0，草莓生长期

1，草莓开花期

2，草莓结果期

3，草莓成熟期


###  题目特点及常用方法

分析赛题，细粒度图像分类 (Fine-grained Image Categorization), 又被称作子类别图像分类 (Sub-category Recognition)。 其目的是对属于同一基础类别的图像进行更加细致的子类划分, 但由于子类别间细微的类间差异以及较大的类内差异, 与传统的图像分类任务相比, 细粒度图像分类难度明显要大很多。从下图中的草莓图像可以看出，不同病变情况的叶子长相非常相似，此外同一类别由于姿态、背景以及拍摄角度的不同，也存在较大的类内差异。



In [ ]:
# 数据集解压

!cd 'data/data97595' && unzip -q nongzuowu.zip

### 数据EDA


&emsp;&emsp;探索性数据分析（Exploratory Data Analysis，简称EDA），是指对已有的数据（原始数据）进行分析探索，通过作图、制表、方程拟合、计算特征量等手段探索数据的结构和规律的一种数据分析方法。一般来说，我们最初接触到数据的时候往往是毫无头绪的，不知道如何下手，这时候探索性数据分析就非常有效。

  对于图像分类任务，我们通常首先应该统计出每个类别的数量，查看训练集的数据分布情况。通过数据分布情况分析赛题，形成解题思路。（洞察数据的本质很重要。）

In [ ]:
import pandas as pd

df = pd.read_csv('data/data97595/nongzuowu/train.csv')
d=df['label'].hist().get_figure()
d.savefig('2.jpg')


![](https://ai-studio-static-online.cdn.bcebos.com/f5986310311f4ff8811e5ef8a21e42bb5d42fac122e84169950d0e38d2ac9af4)


## baseline搭建

### 数据预处理

In [6]:
import numpy as np
import cv2
import os
 
img_h, img_w = 224, 224   #根据自己数据集适当调整，影响不大
means, stdevs = [], []
img_list = []
 
imgs_path = 'data/data97595/nongzuowu/test/testA'
imgs_path_list = os.listdir(imgs_path)
 
len_ = len(imgs_path_list)
i = 0
for item in imgs_path_list:
    img = cv2.imread(os.path.join(imgs_path,item))
    img = cv2.resize(img,(img_w,img_h))
    img = img[:, :, :, np.newaxis]
    img_list.append(img)
    i += 1
    # print(i,'/',len_)

imgs_path = 'data/data97595/nongzuowu/test/testA'
imgs_path_list = os.listdir(imgs_path)
 
len_ = len(imgs_path_list)
i = 0
for item in imgs_path_list:
    img = cv2.imread(os.path.join(imgs_path,item))
    img = cv2.resize(img,(img_w,img_h))
    img = img[:, :, :, np.newaxis]
    img_list.append(img)
    i += 1
    # print(i,'/',len_)

# print(len(img_list))

imgs = np.concatenate(img_list, axis=3)
imgs = imgs.astype(np.float32) / 255.
 
for i in range(3):
    pixels = imgs[:, :, i, :].ravel()  # 拉成一行
    means.append(np.mean(pixels))
    stdevs.append(np.std(pixels))
 
# BGR --> RGB ， CV读取的需要转换，PIL读取的不用转换
means.reverse()
stdevs.reverse()
 
print("normMean = {}".format(means))
print("normStd = {}".format(stdevs))

normMean = [0.4494708, 0.45852903, 0.29217097]
normStd = [0.2794664, 0.2744329, 0.2591805]


In [1]:
# 导入所需要的库
from sklearn.utils import shuffle
import os
import pandas as pd
import numpy as np
from PIL import Image

import paddle
import paddle.nn as nn
from paddle.io import Dataset
import paddle.vision.transforms as T
import paddle.nn.functional as F
from paddle.metric import Accuracy

import warnings
warnings.filterwarnings("ignore")

# 读取数据
train_images = pd.read_csv('data/data97595/nongzuowu/train.csv')

# labelshuffling

def labelShuffling(dataFrame, groupByName = 'label'):

    groupDataFrame = dataFrame.groupby(by=[groupByName])
    labels = groupDataFrame.size()
    print("length of label is ", len(labels))
    maxNum = max(labels)
    lst = pd.DataFrame()
    for i in range(len(labels)):
        print("Processing label  :", i)
        tmpGroupBy = groupDataFrame.get_group(i)
        createdShuffleLabels = np.random.permutation(np.array(range(maxNum))) % labels[i]
        print("Num of the label is : ", labels[i])
        lst=lst.append(tmpGroupBy.iloc[createdShuffleLabels], ignore_index=True)
        print("Done")
    # lst.to_csv('test1.csv', index=False)
    return lst

# 划分训练集和校验集
all_size = len(train_images)
# print(all_size)
train_size = int(all_size * 0.85)
train_image_list = train_images[:train_size]
val_image_list = train_images[train_size:]

df = labelShuffling(train_image_list)
df = shuffle(df)

train_image_path_list = df['image'].values
label_list = df['label'].values
label_list = paddle.to_tensor(label_list, dtype='int64')
train_label_list = paddle.nn.functional.one_hot(label_list, num_classes=4)

val_image_path_list = val_image_list['image'].values
val_label_list = val_image_list['label'].values
val_label_list = paddle.to_tensor(val_label_list, dtype='int64')
val_label_list = paddle.nn.functional.one_hot(val_label_list, num_classes=4)

# 定义数据预处理
data_transforms = T.Compose([
    T.Resize(size=(224, 224)),
    T.RandomHorizontalFlip(0.5),
    T.RandomVerticalFlip(0.5),
    T.RandomRotation(30),
    T.Transpose(),    # HWC -> CHW
    T.Normalize(
        mean=[0.44381204, 0.46818522, 0.30317423],        # 归一化
        std=[0.27267984, 0.26876768, 0.25462458],
        to_rgb=True)    
])



length of label is  4
Processing label  : 0
Num of the label is :  132
Done
Processing label  : 1
Num of the label is :  119
Done
Processing label  : 2
Num of the label is :  119
Done
Processing label  : 3
Num of the label is :  103
Done


In [2]:
# 构建Dataset
class MyDataset(paddle.io.Dataset):
    """
    步骤一：继承paddle.io.Dataset类
    """
    def __init__(self, train_img_list, val_img_list,train_label_list,val_label_list, mode='train'):
        """
        步骤二：实现构造函数，定义数据读取方式，划分训练和测试数据集
        """
        super(MyDataset, self).__init__()
        self.img = []
        self.label = []
        # 借助pandas读csv的库
        self.train_images = train_img_list
        self.test_images = val_img_list
        self.train_label = train_label_list
        self.test_label = val_label_list
        if mode == 'train':
            # 读train_images的数据
            for img,la in zip(self.train_images, self.train_label):
                self.img.append('data/data97595/nongzuowu/train/'+img)
                self.label.append(la)
        else:
            # 读test_images的数据
            for img,la in zip(self.test_images, self.test_label):
                self.img.append('data/data97595/nongzuowu/train/'+img)
                self.label.append(la)

    def load_img(self, image_path):
        # 实际使用时使用Pillow相关库进行图片读取即可，这里我们对数据先做个模拟
        image = Image.open(image_path).convert('RGB')
        image = np.asarray(image)
        image = image.astype(np.float32) / 255.
        return image

    def __getitem__(self, index):
        """
        步骤三：实现__getitem__方法，定义指定index时如何获取数据，并返回单条数据（训练数据，对应的标签）
        """
        image = self.load_img(self.img[index])
        label = self.label[index]
        # label = paddle.to_tensor(label)
        
        return data_transforms(image), paddle.nn.functional.label_smooth(label)

    def __len__(self):
        """
        步骤四：实现__len__方法，返回数据集总数目
        """
        return len(self.img)

In [3]:
#train_loader
train_dataset = MyDataset(train_img_list=train_image_path_list, val_img_list=val_image_path_list, train_label_list=train_label_list, val_label_list=val_label_list, mode='train')
train_loader = paddle.io.DataLoader(train_dataset, places=paddle.CPUPlace(), batch_size=196, shuffle=True, num_workers=0)

#val_loader
val_dataset = MyDataset(train_img_list=train_image_path_list, val_img_list=val_image_path_list, train_label_list=train_label_list, val_label_list=val_label_list, mode='test')
val_loader = paddle.io.DataLoader(val_dataset, places=paddle.CPUPlace(), batch_size=196, shuffle=True, num_workers=0)

In [ ]:


print('=============train dataset=============')
for image, label in train_dataset:
    print('image shape: {}, label: {}'.format(image.shape, label))
    break



### 模型训练

In [ ]:
# # 调用飞桨框架的VisualDL模块，保存信息到目录中。
# callback = paddle.callbacks.VisualDL(log_dir='visualdl_log_dir')

# from visualdl import LogReader, LogWriter

# args={
#     'logdir':'./vdl',
#     'file_name':'vdlrecords.model.log',
#     'iters':0,
# }

# # 配置visualdl
# write = LogWriter(logdir=args['logdir'], file_name=args['file_name'])
# #iters 初始化为0
# iters = args['iters'] 

# #自定义Callback
# class Callbk(paddle.callbacks.Callback):
#     def __init__(self, write, iters=0):
#         self.write = write
#         self.iters = iters

#     def on_train_batch_end(self, step, logs):

#         self.iters += 1

#         #记录loss
#         self.write.add_scalar(tag="loss",step=self.iters,value=logs['loss'][0])
#         #记录 accuracy
#         self.write.add_scalar(tag="acc",step=self.iters,value=logs['acc'])

In [4]:
from work.res2net import Res2Net50_vd_26w_4s

# 模型封装
model_res = Res2Net50_vd_26w_4s(class_dim=4)
model = paddle.Model(model_res)

# 定义优化器

# scheduler = paddle.optimizer.lr.LinearWarmup(
#         learning_rate=0.5, warmup_steps=20, start_lr=0, end_lr=0.5, verbose=True)
# optim = paddle.optimizer.SGD(learning_rate=scheduler, parameters=model.parameters())
optim = paddle.optimizer.Adam(learning_rate=3e-4, parameters=model.parameters())

# 配置模型
model.prepare(
    optim,
    paddle.nn.CrossEntropyLoss(soft_label=True),
    Accuracy()
    )

model.load('work/Res2Net50_vd_26w_4s_pretrained.pdparams',skip_mismatch=True)

# 模型训练与评估
model.fit(train_loader,
        val_loader,
        log_freq=1,
        epochs=10,
        # callbacks=Callbk(write=write, iters=iters),
        verbose=1,
        )

The loss value printed in the log is the current step, and the metric is the average value of previous steps.
Epoch 1/10
step 3/3 [==============================] - loss: 0.5436 - acc: 0.6837 - 3s/step        
Eval begin...
step 1/1 [==============================] - loss: 0.5187 - acc: 0.9167 - 1s/step
Eval samples: 84
Epoch 2/10
step 3/3 [==============================] - loss: 0.4956 - acc: 0.9754 - 3s/step        
Eval begin...
step 1/1 [==============================] - loss: 0.4394 - acc: 0.9762 - 1s/step
Eval samples: 84
Epoch 3/10
step 3/3 [==============================] - loss: 0.4067 - acc: 0.9962 - 3s/step        
Eval begin...
step 1/1 [==============================] - loss: 0.4317 - acc: 1.0000 - 1s/step
Eval samples: 84
Epoch 4/10
step 3/3 [==============================] - loss: 0.3779 - acc: 0.9886 - 3s/step        
Eval begin...
step 1/1 [==============================] - loss: 0.4196 - acc: 0.9881 - 1s/step
Eval samples: 84
Epoch 5/10
step 3/3 [=====================

In [5]:
# 保存模型参数
# model.save('Hapi_MyCNN')  # save for training
model.save('Hapi_MyCNN2', False)  # save for inference

### 模型预测

In [7]:
import os, time
import matplotlib.pyplot as plt
import paddle
from PIL import Image
import numpy as np
import patta as tta


use_gpu = True
model_file_path="Hapi_MyCNN2"
paddle.set_device('gpu:0') if use_gpu else paddle.set_device('cpu')
model = paddle.jit.load(model_file_path)

model = tta.ClassificationTTAWrapper(model, tta.aliases.ten_crop_transform(224,224))
model.eval() #训练模式

def load_image(img_path):
    '''
    预测图片预处理
    '''
    img = Image.open(img_path).convert('RGB')
    
    #resize
    img = img.resize((224, 224), Image.BILINEAR) #Image.BILINEAR双线性插值
    img = np.array(img).astype('float32')

    # HWC to CHW 
    img = img.transpose((2, 0, 1))
    
    #Normalize
    img = img / 255         #像素值归一化
    # print(img)
    mean = [0.4494708, 0.45852903, 0.29217097]   
    std = [0.2794664, 0.2744329, 0.2591805]
    img[0] = (img[0] - mean[0]) / std[0]
    img[1] = (img[1] - mean[1]) / std[1]
    img[2] = (img[2] - mean[2]) / std[2]
    
    return img

def infer_img(path, model):
    '''
    模型预测
    '''
    

    #对预测图片进行预处理
    infer_imgs = []
    infer_imgs.append(load_image(path))
    infer_imgs = np.array(infer_imgs)
    label_list = ['0:優良', '1:良', '2:加工品', '3:規格外']
    label_pre = []
    for i in range(len(infer_imgs)):
        data = infer_imgs[i]
        dy_x_data = np.array(data).astype('float32')
        dy_x_data = dy_x_data[np.newaxis,:, : ,:]
        img = paddle.to_tensor(dy_x_data)
        out = model(img)
        # print(out[0])
        # print(paddle.nn.functional.softmax(out)[0]) # 若模型中已经包含softmax则不用此行代码。

        lab = np.argmax(out.numpy())  #argmax():返回最大数的索引
        label_pre.append(lab)
       
    return label_pre


In [8]:
img_list = os.listdir('data/data97595/nongzuowu/test/testA/')
img_list.sort(key=lambda x: int(x[5:-4]))
# img_list

In [9]:
pre_list = []

for i in range(len(img_list)):
    pre_list.append(infer_img(path='data/data97595/nongzuowu/test/testA/' + img_list[i], model=model)[0])

In [10]:
import pandas as pd

img = pd.DataFrame(img_list)
img = img.rename(columns = {0:"image_id"})
img['category_id'] = pre_list

img.to_csv('submit123456.csv', index=False)

# 提交结果

![](https://ai-studio-static-online.cdn.bcebos.com/3a888bfd6634402e9ae1f42e8b3a9bdf1d3104c9346a4fba9ea51ca8146185ed)
